# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import gc
import os
import pandas as pd
import numpy as np

from src import CTX, SEED, FOLDERS
from src.data import io
from src.features import COLUMNS, build_features

In [3]:
observations = io.load_data(FOLDERS.PROCESSED, CTX + 'observations', pd.DataFrame())
observations

Loading ../data/processed/CC-Labs-hv_observations.h5


,Observation
0,"Tens of categories with very few items, a hand..."
1,First word of category name a good candidate f...
2,(Categories already sorted on category name)
3,First word of shop name a good candidate for s...
4,(Shops already sorted on shop name)
5,Submissions are evaluated by root mean squared...
6,"All shop, item, and category references are va..."
7,Training data does not have explicit NULLs
8,For each ID in the test set (shop_id/item_id c...
9,One in seven test data keys do not have entrie...


# Read raw data

In [8]:
from src.data import io
sales, test, items, categories, shops = io.read_raw(FOLDERS.RAW)

In [9]:
### TODO
sales = sales.loc[sales.shop_id <= 10]
test = test.loc[test.shop_id <= 10]
### TODO

In [10]:
train = build_features.rollup_and_clip_sales(sales)
train.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


# Create grid

In [11]:
max_train_date_block_num = train.date_block_num.max()
max_train_date_block_num

33

In [12]:
from tqdm import tqdm_notebook
all_data = build_features.create_grid(
    pd.concat([
        train.rename(columns={'item_cnt_month' : 'target'}),
        test.assign(date_block_num=max_train_date_block_num+1).drop(columns=['ID'])],
        ignore_index=True, sort=False),
    shops, items, categories,
    COLUMNS.KEYS_AND_TIME,
    tqdm_notebook)
print(len(all_data))
all_data.head()

/home/mortennp/miniconda3/envs/comp-ds-3_6/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


710837


,date_block_num,shop_id,item_id,target,super_shop_id,item_category_id,super_category_id,target_shop,target_super_shop,target_item,target_category,target_super_category
0,0,0,32,6.0,0,40,7,5411.0,8343.0,27.0,5216.0,6641.0
1,0,0,33,3.0,0,37,7,5411.0,8343.0,8.0,933.0,6641.0
2,0,0,35,1.0,0,40,7,5411.0,8343.0,5.0,5216.0,6641.0
3,0,0,43,1.0,0,40,7,5411.0,8343.0,1.0,5216.0,6641.0
4,0,0,51,2.0,0,57,9,5411.0,8343.0,3.0,195.0,2193.0


In [13]:
io.save_data(FOLDERS.INTERIM, CTX + 'grid', all_data)
del all_data
gc.collect()

Saving ../data/interim/CC-Labs-hv_grid.h5


40

# Create lags

In [14]:
all_data = io.load_data(FOLDERS.INTERIM, CTX + 'grid', pd.DataFrame())

Loading ../data/interim/CC-Labs-hv_grid.h5


In [15]:
lagged_data, to_drop_cols = build_features.create_lags(
    all_data,
    COLUMNS.KEYS_AND_TIME + COLUMNS.DERIVED_KEYS,
    tqdm_notebook)
lagged_data.head()

,date_block_num,shop_id,item_id,target,super_shop_id,item_category_id,super_category_id,target_shop,target_super_shop,target_item,...,target_item_lag_5,target_shop_lag_5,target_super_category_lag_5,target_super_shop_lag_5,target_lag_12,target_category_lag_12,target_item_lag_12,target_shop_lag_12,target_super_category_lag_12,target_super_shop_lag_12
0,0,0,32,6.0,0,40,7,5411.0,8343.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,33,3.0,0,37,7,5411.0,8343.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,35,1.0,0,40,7,5411.0,8343.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,43,1.0,0,40,7,5411.0,8343.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,51,2.0,0,57,9,5411.0,8343.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
to_drop_cols = pd.DataFrame(to_drop_cols, columns=['col_name'])
to_drop_cols.head()

,col_name
0,target_super_category
1,target_category
2,target_item
3,target_shop
4,target_super_shop


In [17]:
del all_data
gc.collect()

7

In [18]:
io.save_data(FOLDERS.INTERIM, CTX + 'lagged_data', lagged_data)
io.save_data(FOLDERS.INTERIM, CTX + 'to_drop_cols', to_drop_cols)
del lagged_data, to_drop_cols
gc.collect()

Saving ../data/interim/CC-Labs-hv_lagged_data.h5
Saving ../data/interim/CC-Labs-hv_to_drop_cols.h5


55

# Encode and normalize

In [19]:
lagged_data = io.load_data(FOLDERS.INTERIM, CTX + 'lagged_data', pd.DataFrame())
to_drop_cols = io.load_data(FOLDERS.INTERIM, CTX + 'to_drop_cols', pd.DataFrame())

Loading ../data/interim/CC-Labs-hv_lagged_data.h5
Loading ../data/interim/CC-Labs-hv_to_drop_cols.h5


In [20]:
numeric_features = sorted(list(set(list(lagged_data.columns.values)) 
                               - set(to_drop_cols.col_name.values) - set(COLUMNS.KEYS_AND_TIME) - set(COLUMNS.DERIVED_KEYS) - set(['item_category_id'])))
print(numeric_features)

['target_category_lag_1', 'target_category_lag_12', 'target_category_lag_2', 'target_category_lag_3', 'target_category_lag_4', 'target_category_lag_5', 'target_item_lag_1', 'target_item_lag_12', 'target_item_lag_2', 'target_item_lag_3', 'target_item_lag_4', 'target_item_lag_5', 'target_lag_1', 'target_lag_12', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_shop_lag_1', 'target_shop_lag_12', 'target_shop_lag_2', 'target_shop_lag_3', 'target_shop_lag_4', 'target_shop_lag_5', 'target_super_category_lag_1', 'target_super_category_lag_12', 'target_super_category_lag_2', 'target_super_category_lag_3', 'target_super_category_lag_4', 'target_super_category_lag_5', 'target_super_shop_lag_1', 'target_super_shop_lag_12', 'target_super_shop_lag_2', 'target_super_shop_lag_3', 'target_super_shop_lag_4', 'target_super_shop_lag_5']


In [21]:
#categorical_features = list(set(index_cols + ['item_category_id']) - set(['date_block_num']))
categorical_features = ['shop_id'] + COLUMNS.DERIVED_KEYS
print(categorical_features)

['shop_id', 'super_shop_id', 'item_category_id', 'super_category_id']


In [27]:
mapper = build_features.create_mapper(categorical_features, numeric_features)
mapped_data = mapper.fit_transform(lagged_data.drop(to_drop_cols.col_name.values, axis=1)).astype(np.float32)
#mapper.transformed_names_

In [28]:
mapped_data.dtype

dtype('float32')

In [29]:
#train = lagged_data.loc[lagged_data.date_block_num <= max_train_date_block_num]
#test_lagged = lagged_data.loc[lagged_data.date_block_num == max_train_date_block_num + 1]
train_indices = lagged_data.date_block_num <= max_train_date_block_num
test_indices = lagged_data.date_block_num == max_train_date_block_num + 1

In [30]:
X_train = mapped_data[train_indices]
X_test = mapped_data[test_indices]

target_col = 'target'
y_train = lagged_data.loc[train_indices, [target_col]].values.ravel()

In [31]:
dates_train=lagged_data.loc[train_indices,['date_block_num']]
#dates_train.head()
predictions = lagged_data.loc[test_indices, ['shop_id', 'item_id']]
#predictions.head()

In [32]:
io.save_data(FOLDERS.PROCESSED, CTX + 'X_train', X_train)
io.save_data(FOLDERS.PROCESSED, CTX + 'X_test', X_test)
io.save_data(FOLDERS.PROCESSED, CTX + 'y_train', y_train)
io.save_data(FOLDERS.PROCESSED, CTX + 'dates_train', dates_train)
io.save_data(FOLDERS.PROCESSED, CTX + 'predictions', predictions)

Saving ../data/processed/CC-Labs-hv_X_train.h5
Saving ../data/processed/CC-Labs-hv_X_test.h5
Saving ../data/processed/CC-Labs-hv_y_train.h5
Saving ../data/processed/CC-Labs-hv_dates_train.h5
Saving ../data/processed/CC-Labs-hv_predictions.h5


In [33]:
del lagged_data
del mapped_data
gc.collect()

94

In [35]:
X_train

array([[ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.1965910e-01, -4.9160501e-01,  3.2000000e+01],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.1965910e-01, -4.9160501e-01,  3.3000000e+01],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.1965910e-01, -4.9160501e-01,  3.5000000e+01],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.1965910e-01, -4.9160501e-01,  2.1682000e+04],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.1965910e-01, -4.9160501e-01,  2.1843000e+04],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -2.5105155e-01, -2.2498226e-01,  2.1929000e+04]], dtype=float32)